In [2]:
pip install depthai


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import depthai as dai
import cv2
import numpy as np

def create_pipeline():
    # Create a pipeline
    pipeline = dai.Pipeline()

    # Define sources and outputs
    cam_left = pipeline.create(dai.node.MonoCamera)
    cam_right = pipeline.create(dai.node.MonoCamera)
    depth = pipeline.create(dai.node.StereoDepth)

    cam_left.setBoardSocket(dai.CameraBoardSocket.LEFT)
    cam_right.setBoardSocket(dai.CameraBoardSocket.RIGHT)
    cam_left.setResolution(dai.MonoCameraProperties.SensorResolution.THE_720_P)
    cam_right.setResolution(dai.MonoCameraProperties.SensorResolution.THE_720_P)

    # Properties
    depth.setConfidenceThreshold(255)
    depth.setSubpixel(False)

    # Linking
    cam_left.out.link(depth.left)
    cam_right.out.link(depth.right)
    xout_depth = pipeline.create(dai.node.XLinkOut)
    xout_depth.setStreamName('depth')
    depth.disparity.link(xout_depth.input)

    return pipeline

# Connect to device and start pipeline
with dai.Device(create_pipeline()) as device:
    # Output queue will be used to get the disparity frames from the outputs defined above
    q_depth = device.getOutputQueue(name="depth", maxSize=4, blocking=False)

    while True:
        in_depth = q_depth.get()  # Blocking call, will wait until a new data has arrived
        frame = in_depth.getFrame()  # Get frame from the output queue
        
        # Normalize the depth map to be in the range of 0-255 for displaying
        frame = (frame * (255 / frame.max())).astype(np.uint8)
        
        cv2.imshow("Depth", frame)

        if cv2.waitKey(1) == ord('q'):
            break

cv2.destroyAllWindows()
